In [23]:
from tqdm import tqdm
import pandas as pd

songs = []
with open("songs/cleaned_songs.txt", 'r') as file:
	for line in file:
		songs.append(int(line))

In [33]:
def isKana(c):
	if 0x3040 <= ord(c) <= 0x309F: return True
	if 0x30A0 <= ord(c) <= 0x30FF: return True
	if 0x31F0 <= ord(c) <= 0x31FF: return True
	return False
def isKanji(c):
	if 0x4E00 <= ord(c) <= 0x9FFF: return True
	return False
def isCharacter(c):
	return isKana(c) or isKanji(c)

freDict = {} # Frequency
posDict = {} # Part of Speech
proDict = {} # Pronunciation
basDict = {} # Basic Form
for id in tqdm(songs):
	with open(f"parsed_lyrics/{id}.txt", 'r') as file:
		for line in file:
			words = line.split()
			if str.isspace(line[0]): continue
			if all(not isCharacter(c) for c in words[0]): continue
			properties = words[1].split(',')
			if properties[6] == '*': continue
			if len(properties) <= 7: print(line)
			name = words[0]
			basicForm = properties[6]
			pos = properties[0]
			if pos in ['助詞', '記号', '助動詞']: continue
			if name not in freDict:
				freDict[name] = 1
				posDict[name] = pos
				proDict[name] = properties[7]
				basDict[name] = basicForm
			else:
				freDict[name] += 1
df = pd.DataFrame({"词频":freDict, "发音": proDict, "词性": posDict, "原型": basDict})
df = df[df["词频"] > 1]
df = df.sort_values(by=["词频"], ascending=False)
print(df)
df.to_csv("歌词高频词.csv", encoding='utf-8-sig')

100%|██████████| 9616/9616 [00:05<00:00, 1657.55it/s]

         词频     发音  词性    原型
君     20265     キミ  名詞     君
し     16665      シ  動詞    する
い     13853      イ  動詞    いる
ん     12258      ン  名詞     ん
の     11599      ノ  名詞     の
...     ...    ...  ..   ...
鳴り響こ      2  ナリヒビコ  動詞  鳴り響く
ベテラン      2   ベテラン  名詞  ベテラン
中庸        2  チュウヨウ  名詞    中庸
ナマ        2     ナマ  名詞    ナマ
外聞        2   ガイブン  名詞    外聞

[26962 rows x 4 columns]
